## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import modules as f # f is for function

%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [3]:
# manufactoring suppliers 
# 'lrcs', 'klac', ''

In [14]:
f.predictions('tsm')

TSM 5m Interval Timestamp: 2024-11-21 03:30:58 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,up,up,static
kelly_1:2.5,0.243827,0.196574,0.196094,0.124528
prob_up,0.326913,0.359623,0.34502,0.25
prob_static,0.353463,0.330146,0.325396,0.375
prob_down,0.319624,0.31023,0.329584,0.375
precision,0.459877,0.426124,0.425781,0.374663
recall,0.576402,0.385659,0.422481,0.537718
f1,0.511588,0.404883,0.424125,0.44162
support,"[516.0, 517.0, 516.0]","[516.0, 517.0, 516.0]","[516.0, 517.0, 516.0]","[516.0, 517.0, 516.0]"


TSM 15m Interval Timestamp: 2024-11-21 03:31:05 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,down,static,down
kelly_1:2.5,0.316092,0.102762,0.316471,0.038261
prob_up,0.133021,0.226513,0.290623,0.25
prob_static,0.520504,0.317471,0.383055,0.0
prob_down,0.346475,0.456016,0.326322,0.75
precision,0.511494,0.359116,0.511765,0.313043
recall,0.517442,0.380117,0.505814,0.210526
f1,0.514451,0.369318,0.508772,0.251748
support,"[170.0, 172.0, 171.0]","[170.0, 172.0, 171.0]","[170.0, 172.0, 171.0]","[170.0, 172.0, 171.0]"


TSM 1h Interval Timestamp: 2024-11-21 03:31:08 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,down,down,down
kelly_1:2.5,0.065517,0.088591,0.026637,0.040979
prob_up,0.330921,0.306771,0.276677,0.25
prob_static,0.350244,0.330368,0.308509,0.25
prob_down,0.318834,0.362861,0.414813,0.5
precision,0.332512,0.348993,0.30474,0.314985
recall,0.297357,0.345898,0.299335,0.228381
f1,0.313953,0.347439,0.302013,0.264781
support,"[454.0, 454.0, 451.0]","[454.0, 454.0, 451.0]","[454.0, 454.0, 451.0]","[454.0, 454.0, 451.0]"


TSM 1wk Interval Timestamp: 2024-11-21 03:31:22 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,static,static,up
kelly_1:2.5,0.030769,0.083636,-0.022472,0.038384
prob_up,0.386787,0.323924,0.335255,0.375
prob_static,0.36667,0.347841,0.346914,0.25
prob_down,0.246544,0.328234,0.317831,0.375
precision,0.307692,0.345455,0.269663,0.313131
recall,0.301075,0.408602,0.258065,0.333333
f1,0.304348,0.374384,0.263736,0.322917
support,"[93.0, 93.0, 92.0]","[93.0, 93.0, 92.0]","[93.0, 93.0, 92.0]","[93.0, 93.0, 92.0]"


TSM 1mo Interval Timestamp: 2024-11-21 03:31:29 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,down
kelly_1:2.5,0.066667,-0.081818,0.030769,-0.076923
prob_up,0.309719,0.233533,0.298797,0.125
prob_static,0.42327,0.48362,0.361117,0.375
prob_down,0.267011,0.282847,0.340086,0.5
precision,0.333333,0.227273,0.307692,0.230769
recall,0.380952,0.238095,0.380952,0.157895
f1,0.355556,0.232558,0.340426,0.1875
support,"[19.0, 21.0, 19.0]","[19.0, 21.0, 19.0]","[19.0, 21.0, 19.0]","[19.0, 21.0, 19.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')